# Research Notebook
## Adam Cherti
## Date: February 2-9, 2026

# 1: Experience
## Describe at least one research activity you worked on this week.

- Implemented sound wave model for dark sector phase transitions using PTArcade
- Fitted both decaying and stable dark sector models to NANOGrav 15yr data
- Created visualization scripts for spectrum and posterior distribution analysis
- Compared gravitational wave signals from different dark sector scenarios

## Motivation:

Building on last week's bubble wall model, this week focused on understanding sound wave contributions to the gravitational wave background from dark sector phase transitions. Phase transitions produce two types of GW sources: bubble wall collisions and sound wave radiation from plasma dynamics. The sound shell model is more realistic than the bubble wall model alone, as real phase transitions generate both sources. By fitting decaying and stable dark sector models separately, we can determine which scenario better explains any signal detected by NANOGrav.

# 2: What? (What happened?)
## Describe what happened during your activities for the week.

I began by studying the sound shell model equations from the LISA 2019 review and the thesis, which parameterizes gravitational wave spectra from sound waves during phase transitions. The key equation is:

$$h^2\Omega_{sw}(f) = R h^2 \tilde{\Omega}_{sw} \left(\frac{\kappa_{sw}\alpha}{\alpha + 1}\right)^2 \left(\frac{\beta}{H}\right)^{-1} y_{sh} s_{sw}(f/f_{p,sw})$$

with shape function $s_{sw}(x) = x^3 \left(\frac{7}{4+3x^2}\right)^{7/2}$ and efficiency factor $y_{sh} = \min[1, \frac{3.38}{\beta/H}\sqrt{\frac{1+\alpha}{\kappa_{sw}\alpha}}]$.

### Initial Approach: Decaying Dark Sector

I first created a sound wave model for a **decaying dark sector** (`model_soundwave.py`) with 5 parameters: α, β/H, T_p (GeV), ε_p, and τ_φ (decay timescale). After fitting this model to NANOGrav 15yr data, I obtained:

- α = 0.202 (moderate phase transition strength)
- β/H = 6.15 (satisfies β/H > 3 percolation requirement)
- T_p = 4.3 × 10⁻⁴ GeV (very cold dark sector phase transition)
- ε_p = 0.159 (≈16% efficiency for sound wave radiation)
- τ_φ = 0.0198 seconds (dark sector decay timescale)

However, **these fitted parameters did not match the predictions shown in the thesis plots**. The model's spectrum did not align with what the thesis expected from similar phase transition scenarios.

### Meeting Discussion and Pivot to Stable Dark Sector

During our meeting, we discussed why the decaying model results diverged from the thesis expectations. Key issues identified:

1. **Model Ambiguity**: The paper is confusing about how to properly model a stable dark sector. The parameterization is unclear.
2. **Too Many Parameters**: The decaying model has 5 free parameters (α, β/H, T_p, ε_p, τ_φ), making it difficult to directly compare with thesis predictions that may not include all these effects.
3. **Parameter Relationships**: It's unclear how some parameters are related and whether certain combinations are physically meaningful.
4. **Physics Clarity**: A stable dark sector (no decay) is simpler and more directly comparable to theoretical predictions.

**Decision**: We decided to focus on the **stable dark sector model** instead, which has only 4 parameters: α, β/H, T_p, and **ξ_p** (the temperature ratio T_dark/T_SM at percolation).

### New Implementation: Stable Dark Sector Model

I created a new model file (`model_soundwave_stable.py`) implementing the sound wave spectrum for a **stable (non-decaying) dark sector**:

$$h^2\Omega_{sw}(f) = R h^2 \tilde{\Omega}_{sw} \left(\frac{\kappa_{sw}\alpha}{\alpha + 1}\right)^2 \left(\frac{\beta}{H}\right)^{-1} y_{sh} s_{sw}(f/f_p)$$

**Key Differences from Decaying Model:**
- **4 parameters instead of 5**: Removes the decay timescale τ_φ
- **ξ_p parameter**: Temperature ratio of dark sector to SM bath at percolation. Defined as ξ_p ≡ T_d,p / T_p
- **Simpler physics**: No decay effects means the GW spectrum depends only on the phase transition dynamics, not on dark sector longevity
- **Better alignment with theory**: Should more closely match thesis predictions since it focuses on core phase transition physics

Parameter ranges:
- α: [10⁻³, 10¹] (phase transition strength)
- β/H: [10⁰, 10³] (inverse duration)
- T_p: [10⁻⁴, 10¹] GeV (temperature at percolation)
- **ξ_p: [10⁻³, 10⁰]** (temperature ratio, cold DS regime favored)

This simpler parameterization with ξ_p as a core physics parameter should provide clearer physical interpretation and better agreement with theoretical expectations.

In [ ]:
#!/usr/bin/env python3
"""
Sound Wave Model for Gravitational Waves from Stable Dark Sector Phase Transitions

This module implements the sound wave contribution to the stochastic gravitational 
wave background (GWB) spectrum from a stable dark sector phase transition.

Based on the sound shell model from the LISA 2019 review and hydrodynamical simulations.
Reference: Equation (4.33) - Sound wave template

Model Parameters for STABLE Dark Sector:
- alpha (α): strength of the phase transition
- beta/H (β/H): inverse duration of the phase transition 
- Tp: SM bath temperature at percolation (in GeV)
- xi_p (ξ_p): temperature ratio = T_dark_sector / T_SM at percolation
"""

import numpy as np
from ptarcade.models_utils import prior

# ======================================================================
# Parameters for Sound Wave Model - STABLE Dark Sector Phase Transition
# ======================================================================

parameters = {
    # alpha: phase transition strength (latent heat / radiation energy)
    # Range: [1e-3, 10] (in log space: -3 to 1)
    "log10_alpha":   prior("Uniform", -3.0, 1.0),
    
    # beta/H: inverse duration of phase transition
    # β/H > 3 required for successful percolation
    # Range: [1, 1e3] (in log space: 0 to 3)
    "log10_betaH":   prior("Uniform", 0.0, 3.0),
    
    # Tp: SM bath temperature at percolation (GeV)
    # Range: [1e-4, 10] GeV (in log space: -4 to 1)
    "log10_Tp_GeV":  prior("Uniform", -4.0, 1.0),
    
    # xi_p: temperature ratio = T_dark_sector / T_SM at percolation
    # Range: [1e-3, 1] (in log space: -3 to 0)
    # Cold dark sector regime: ξ_p ≪ 1 (favored by data)
    "log10_xi_p": prior("Uniform", -3.0, 0.0),
}


def s_sw(x):
    """
    Shape function for sound wave spectrum (sound shell model).
    
    s_sw(x) = x^3 * (7 / (4 + 3*x^2))^(7/2)
    
    This describes the frequency dependence of the sound wave contribution,
    peaking at x ~ 1 (i.e., f ~ f_p).
    """
    numerator = 7.0
    denominator = 4.0 + 3.0 * x**2
    return x**3 * (numerator / denominator)**(7.0 / 2.0)


def y_sh(alpha, betaH, kappa_sw=1.0):
    """
    Efficiency factor for sound shell radiation (stable dark sector).
    
    y_sh = min [1, 3.38 / (β/H) * sqrt((1+α) / (κ_sw*α))]
    
    This factor accounts for the efficiency of converting potential energy
    into gravitational radiation through sound shell dynamics.
    """
    term = 3.38 / betaH * np.sqrt((1.0 + alpha) / (kappa_sw * alpha))
    return min(1.0, term)


def spectrum(
    f,                 # frequencies [Hz] at which PTArcade evaluates the spectrum
    log10_alpha,
    log10_betaH,
    log10_Tp_GeV,
    log10_xi_p,
    **kwargs
):
    """
    Gravitational wave energy density spectrum from sound waves in stable dark sector phase transitions.
    
    h^2 * Ω_sw(f) = R*h^2 * Ω̃_sw * (κ_sw*α / (α+1))^2 * (β/H)^(-1) * y_sh * s_sw(f/f_p)
    
    where:
    - R*h^2 is the overall amplitude factor
    - Ω̃_sw = 0.072 (from hydrodynamical simulations)
    - s_sw(x) = x^3 * (7/(4+3x^2))^(7/2) (sound shell shape function)
    - y_sh = min[1, 3.38/(β/H) * sqrt((1+α)/(κ_sw*α))] (efficiency factor)
    - f_p^em/β = 0.53 (peak frequency relation)
    """
    
    # Decode parameters from log space
    alpha       = 10.0**log10_alpha
    betaH       = 10.0**log10_betaH
    Tp_GeV      = 10.0**log10_Tp_GeV       # SM bath temperature at percolation (GeV)
    xi_p        = 10.0**log10_xi_p         # Temperature ratio (dark sector / SM)
    
    # ======================================================================
    # Convert physics parameters to frequency and amplitude
    # ======================================================================
    
    # Hubble parameter at temperature T_p
    # H(T) ≈ T^2 / (1.66 * sqrt(g_*) * M_Pl)
    
    M_Pl = 1.22e19  # Planck mass in GeV
    g_star = 100.0  # Effective degrees of freedom
    
    H_Tp_GeV = (Tp_GeV**2) / (1.66 * np.sqrt(g_star) * M_Pl)
    
    # Convert Hubble parameter to Hz
    GeV_to_Hz = 1.519e24  # Conversion factor from GeV to Hz
    H_Tp_Hz = H_Tp_GeV * GeV_to_Hz
    
    # Peak frequency from phase transition parameters
    # Using f_p^em/β = 0.53, so f_p = 0.53 * β * H(T_p) / (2π)
    f_em_p_over_beta = 0.53
    f_p = f_em_p_over_beta * betaH * H_Tp_Hz / (2.0 * np.pi)
    
    # Ensure f_p is positive and reasonable
    if f_p <= 0 or np.isnan(f_p) or np.isinf(f_p):
        f_p = 1e-8  # Default fallback frequency
    
    # ======================================================================
    # Amplitude calculation for stable dark sector
    # ======================================================================
    
    # Physical constants from hydrodynamical simulations and LISA 2019 review
    kappa_sw = 1.0                     # Sound wave efficiency (high-velocity limit)
    Omega_tilde_sw = 0.072             # Normalized amplitude from simulations
    
    # For cold dark sector (ξ_p ≪ 1), amplitude scales as Rh2 = ξ_p^(-1)
    if xi_p > 0:
        Rh2 = xi_p**(-1.0)  # Cold dark sector enhancement
    else:
        Rh2 = 1.0
    
    # Amplitude prefactor
    amp = (
        Rh2
        * Omega_tilde_sw
        * (kappa_sw * alpha / (alpha + 1.0))**2
        * (1.0 / betaH)
    )
    
    # ======================================================================
    # Compute spectrum
    # ======================================================================
    
    # Efficiency factor for sound shell radiation
    y = y_sh(alpha, betaH, kappa_sw)
    
    # Dimensionless frequency
    x = f / f_p if f_p > 0 else f / 1e-8
    
    # Protect against invalid values
    spectrum_result = amp * y * s_sw(x)
    
    # Handle any NaN or Inf values
    spectrum_result = np.where(np.isfinite(spectrum_result), spectrum_result, 1e-30)
    
    return spectrum_result

# 3: So what? (What does it mean?)
## Describe your results and insights

### Initial Attempt - Why the Decaying Model Didn't Work

While the decaying dark sector model successfully fit to NANOGrav data mathematically, the fitted parameters didn't align with theoretical expectations from the thesis. This revealed an important lesson:

**Fitting Parameters ≠ Physical Truth**: Just because a model can fit the data doesn't mean the fitted parameters represent realistic physics. The decaying model's parameter combinations may have been degenerate or inconsistent with the theoretical framework the thesis was built on.

### Insights from Model Comparison

By attempting both the decaying and stable dark sector models, we learned several key things:

1. **Model Complexity Matters**: Adding extra parameters (like τ_φ for decay) can improve fit quality but may obscure the underlying physics if those parameters aren't well-constrained by the data or theory.

2. **Parametrization is Critical**: The choice of parameters (ε_p vs ξ_p, for instance) changes how we interpret results. The temperature ratio ξ_p is more directly tied to dark sector properties than an abstract efficiency factor ε_p.

3. **Stability of Physical Interpretation**: The stable dark sector model's 4 parameters directly correspond to core phase transition physics:
   - α: Phase transition strength
   - β/H: Duration and dynamics
   - T_p: Temperature scale
   - ξ_p: Dark sector temperature (direct probe of dark sector properties)

4. **Paper Clarity**: The thesis paper itself is somewhat ambiguous about stable dark sector modeling, which indicates this is an ongoing area of research where different approaches coexist. Our goal is to implement the physically clearest approach.

### Next Steps with Stable Model

By focusing on the stable dark sector model with ξ_p parameterization, we expect to:
- Get clearer physical interpretation of fitted parameters
- Better alignment with thesis theoretical predictions
- Ability to directly constrain dark sector temperature from NANOGrav data
- Foundation for comparing with other dark sector scenarios

# 4. Now what? (What's next?)
## Plan for the next week

After meeting with my advisor and discussing the discrepancies between the decaying model results and thesis expectations, we pivoted to focus on the **stable dark sector model**. 

**Next Week's Objectives:**
1. **Fit stable dark sector model** to NANOGrav 15yr data using PTArcade
   - Command: `ptarcade -m model_soundwave_stable.py`
   - Generate MCMC chains with proper convergence diagnostics
   
2. **Generate corner plots** showing posterior distributions for {α, β/H, ξ_p, T_p}
   - Extract 68% (1σ) and 95% (2σ) credible intervals
   - Visualize parameter correlations
   
3. **Compare stable vs decaying models**
   - Extract best-fit parameters from both models
   - Create overlay plot showing spectrum predictions from each
   - Assess which better aligns with thesis ex

4. **Calculate Bayes factors** (if time permits)
   - Quantitatively compare model evidence: decaying vs stable vs bubble wall
   - Determine which scenario is statistically favored by NANOGrav observations


# 5. Bibliography

[1] NANOGrav Collaboration, *The NANOGrav 15-year Data Release,* (2023) [arXiv:2306.16221 [astro-ph.HE]].

[2] Carlo Tasillo, *Gravitational Waves from Phase Transitions in the Early Universe,* Doctoral Thesis, Universität Hamburg (2022). Available at: https://ediss.sub.uni-hamburg.de/bitstream/ediss/11329/1/PhD_thesis_Carlo_Tasillo_Bibversion.pdf

[3] PTArcade Documentation and GitHub Repository: https://github.com/andrea-mitridate/PTArcade


# 6. Technical Details: Sound Wave vs Bubble Wall Models

## Overview

During a first-order phase transition, two primary mechanisms produce gravitational waves:

1. **Bubble Wall Collisions** - Direct collision energy from expanding bubbles
2. **Sound Waves** - Plasma dynamics and pressure waves from bubble expansion

## Sound Wave Model Equation

The sound wave spectrum is:

$$h^2\Omega_{sw}(f) = \varepsilon_p \tilde{\Omega}_{sw} \left(\frac{\kappa_{sw}\alpha}{\alpha + 1}\right)^2 \left(\frac{\beta}{H}\right)^{-1} y_{sh} s_{sw}(f/f_p)$$

where:
- **$\tilde{\Omega}_{sw} = 0.072$** - Normalized amplitude from hydrodynamical simulations: $\tilde{\Omega}_{sw} = 3 \times 0.012 \times 0.687 \times (8\pi)^{1/3}$
- **$s_{sw}(x) = x^3 \left(\frac{7}{4+3x^2}\right)^{7/2}$** - Sound shell shape function
- **$y_{sh} = \min[1, \frac{3.38}{\beta/H}\sqrt{\frac{1+\alpha}{\kappa_{sw}\alpha}}]$** - Efficiency factor accounting for sound shell lifetime
- **$f_p^{em}/\beta = 0.53$** - Peak frequency relation from simulations
- **$\kappa_{sw} \approx 1$** - High-velocity (runaway) bubble wall efficiency

## Key Physical Differences: Sound Waves vs Bubble Walls

| Feature | Bubble Walls | Sound Waves |
|---------|-------------|-------------|
| Shape function | $s_{bw}(x) = x^{2.8}/(1+2.8x^{3.8})$ | $s_{sw}(x) = x^3(7/(4+3x^2))^{7/2}$ |
| Efficiency scaling | $(\beta/H)^{-1}$ | $(\beta/H)^{-1}$ (same) |
| Duration sensitivity | Direct GW lifetime | Lifetime via $y_{sh}$ suppression |
| Peak frequency | $f_p^{bw}$ (different relation) | $f_p^{em}/\beta = 0.53$ |
| Physical origin | Collision energy | Plasma shock dynamics |
| Dominance condition | $\kappa_\phi \gg \kappa_{sw}$ | $\kappa_{sw} > \kappa_\phi$ |

## Decaying vs Stable Dark Sector

**Decaying Dark Sector:**
- Dark sector eventually decays with lifetime τ_φ
- 5 parameters: α, β/H, T_p, ε_p, τ_φ
- Amplitude suppression at early times due to decay
- More complex physics but more realistic for some models

**Stable Dark Sector:**
- Dark sector persists indefinitely after phase transition
- 4 parameters: α, β/H, T_p, ε_p
- Simpler physics (no decay effects)
- Relevant for scenarios with stable dark matter candidates

## Observational Implications

By fitting both models to NANOGrav data, we can:
1. **Determine which scenario is favored** using Bayes factors
2. **Extract dark sector constraints** on temperature, phase transition strength, and lifetime
3. **Test new physics** - deviations from standard expectations would indicate exotic dark sector properties